In [2]:
import os 
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(device)

'NVIDIA RTX A1000 Laptop GPU'

In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.7.1+cu128
12.8
True


In [45]:
import torch 
import datasets

from datasets import load_dataset
from transformers import AutoTokenizer

model_name="google/bert_uncased_L-2_H-128_A-2"

batch_size = 32
tokenizer = AutoTokenizer.from_pretrained(model_name)

cola_dataset = load_dataset("glue", "cola")

In [149]:
cola_dataset = load_dataset("glue", "cola")
cola_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [46]:
train_data = cola_dataset["train"]
val_data = cola_dataset["validation"]
len(train_data), len(val_data)

(8551, 1043)

In [47]:
train_data, val_data

(Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 8551
 }),
 Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 1043
 }))

In [147]:
len(train_data["sentence"])
train_data["sentence"][:3]

["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up."]

In [148]:
def tokenize_data(example):
    print(f"Processing batch of size: {len(example['sentence'])}")
    print(f"Processing batch of size: {(example['sentence'][:3])}")
    return tokenizer(
        example["sentence"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

train_data1 = train_data.map(tokenize_data, batched=True)
train_data1

Map:  12%|█▏        | 1000/8551 [00:00<00:01, 6490.26 examples/s]

Processing batch of size: 1000
Processing batch of size: ["Our friends won't buy this analysis, let alone the next one we propose.", "One more pseudo generalization and I'm giving up.", "One more pseudo generalization or I'm giving up."]
Processing batch of size: 1000
Processing batch of size: ['I read three his books.', 'I read every his book.', 'I read his every book.']


Map:  35%|███▌      | 3000/8551 [00:00<00:00, 6457.56 examples/s]

Processing batch of size: 1000
Processing batch of size: ['The noise gave Terry a headache.', 'The noise gave a headache to Terry.', 'Jill threw the ball from home plate to third base.']
Processing batch of size: 1000
Processing batch of size: ['Paintings admire easily.', 'I admired him as a teacher.', 'I admired him a teacher.']


Map:  58%|█████▊    | 5000/8551 [00:00<00:00, 6733.93 examples/s]

...

Map:  82%|████████▏ | 7000/8551 [00:01<00:00, 6694.12 examples/s]

Processing batch of size: 1000
Processing batch of size: ['I should not eat plums.', 'I have not should eat plums.', 'I must can eat plums.']
Processing batch of size: 1000
Processing batch of size: ['John sang beautifully.', 'John sang a carol.', 'John sang beautifully and a carol.']


Map: 100%|██████████| 8551/8551 [00:01<00:00, 6593.17 examples/s]

Processing batch of size: 551
Processing batch of size: ['Who did drink the poison?', "Evan's every idea was completely insane.", 'Sally is making scones, and Gillian is too.']


Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8551
})

In [77]:
train_data1, type(train_data1)

(Dataset({
     features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 8551
 }),
 datasets.arrow_dataset.Dataset)

In [87]:
train_data1.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [88]:
train_data1, type(train_data1)

(Dataset({
     features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 8551
 }),
 datasets.arrow_dataset.Dataset)

In [91]:
train_data1[0].keys(), train_data1[1].keys()

(dict_keys(['label', 'input_ids', 'attention_mask']),
 dict_keys(['label', 'input_ids', 'attention_mask']))

In [93]:
train_data[0].keys()

dict_keys(['sentence', 'label', 'idx'])

In [96]:
train_data1[0].keys()

dict_keys(['label', 'input_ids', 'attention_mask'])

In [97]:
train_loader = torch.utils.data.DataLoader(train_data1, batch_size=batch_size)

In [100]:
len(train_loader), 268*32, len(train_data)

(268, 8576, 8551)

In [118]:
train_iter = iter(train_loader)

In [141]:
next(train_iter)["input_ids"]

tensor([[  101,  2507,  1996,  ...,     0,     0,     0],
        [  101,  2507,  1996,  ...,     0,     0,     0],
        [  101, 14548,  1996,  ...,     0,     0,     0],
        ...,
        [  101,  2017,  4521,  ...,     0,     0,     0],
        [  101,  2027,  2409,  ...,     0,     0,     0],
        [  101,  3044, 13190,  ...,     0,     0,     0]])

"Our friends won't buy this analysis, let alone the next one we propose."

In [283]:
def tokenize_data(example):
    print(f"Processing batch of size: {len(example['sentence'])}")
    return tokenizer(
        example["sentence"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

text = train_data[0]["sentence"]
inference_sample = {"sentence": text}
print(type(inference_sample))
tokenize_data(inference_sample)


<class 'dict'>
Processing batch of size: 71


{'input_ids': [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [279]:
train_data, type(train_data)

(Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 8551
 }),
 datasets.arrow_dataset.Dataset)

In [56]:
train_data, train_data1

(Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 8551
 }),
 Dataset({
     features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 8551
 }))

In [59]:
len(train_data1[0]["input_ids"]), len(train_data1[0]["token_type_ids"]), len(train_data1[0]["attention_mask"])

(512, 512, 512)

In [58]:
sentence = "Our friends won't buy this analysis, let alone the next one we propose."
tokens = tokenizer.tokenize(sentence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(len(tokens), len(ids))

17 17


In [69]:
train_data1[0]["attention_mask"][:20]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [75]:
ids_to_decode = train_data1[0]["input_ids"]
print(ids_to_decode[:20])
tokenizer.decode(ids_to_decode[:20])

[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102, 0]


"[CLS] our friends won't buy this analysis, let alone the next one we propose. [SEP] [PAD]"

In [158]:
tokenizer.decode(train_data1[0]["input_ids"])

"[CLS] our friends won't buy this analysis, let alone the next one we propose. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [277]:
len(train_loader), len(val_loader)

NameError: name 'val_loader' is not defined

##### model

In [271]:
import pytorch_lightning as pl
from transformers import AutoModel

class ColaModel(pl.LightningModule):
    def __init__(self, model_name="google/bert_uncased_L-2_H-128_A-2", lr=1e-2):
        super().__init__()

        # stores all the provided args under the "self.hparams" attribute
        self.save_hyperparameters() 
        # equivalent
        # self.save_hyperparameters("model_name", "lr")
        # self.save_hyperparameters("model_name")
        # accessible via 
        self.hparams.model_name

model = ColaModel()
# model.hparams.model_name, model.hparams.lr
model.hparams.model_name

'google/bert_uncased_L-2_H-128_A-2'

In [274]:
model.hparams.lr, model.hparams["lr"]

(0.01, 0.01)

In [170]:
bert = AutoModel.from_pretrained(model_name)
bert.config

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.53.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [172]:
bert.config.hidden_size

128

In [193]:
for batch in train_loader:
    print(batch.keys())
    print(type(batch))
    print(type(batch["input_ids"]))
    print(type(batch["attention_mask"]))
    sample = batch
    break

dict_keys(['label', 'input_ids', 'attention_mask'])
<class 'dict'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [296]:
type(sample["input_ids"])

torch.Tensor

In [297]:
outputs = bert(input_ids = sample["input_ids"], attention_mask = sample["attention_mask"])
print(outputs.last_hidden_state.shape) # (bs, seq_length, hidden_dim)
type(outputs)

len(dir(outputs)), vars(outputs).keys()

torch.Size([32, 512, 128])


(61,
 dict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'past_key_values', 'attentions', 'cross_attentions']))

In [264]:
torch.manual_seed(32)
a = torch.Tensor(2,2)
b = torch.randn(2,2)
print(a)
print(b)

_, preds = torch.max(a, dim=1)
preds, preds.cpu()

tensor([[0.8651, 0.0284],
        [0.5256, 0.3633]])
tensor([[ 0.8651,  0.0284],
        [ 0.5256, -0.3633]])


(tensor([0, 0]), tensor([0, 0]))

In [253]:
from sklearn.metrics import accuracy_score

y_true = [0, 1, 1, 0]
y_pred = [1, 1, 0, 0]

accuracy = accuracy_score(y_true, y_pred)
print(accuracy)  # Output: 0.5

0.5


In [254]:
accuracy = accuracy_score(y_pred, y_true)
print(accuracy)  # Output: 0.5

0.5


In [316]:
for batch in train_loader:
    sample = batch
    break

sample, 
# sample["input_ids"][0], sample["attention_mask"][0]

({'label': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
          1, 0, 0, 1, 1, 1, 1, 1]),
  'input_ids': tensor([[  101,  2256,  2814,  ...,     0,     0,     0],
          [  101,  2028,  2062,  ...,     0,     0,     0],
          [  101,  2028,  2062,  ...,     0,     0,     0],
          ...,
          [  101,  5965, 12808,  ...,     0,     0,     0],
          [  101,  2198, 10948,  ...,     0,     0,     0],
          [  101,  3021, 24471,  ...,     0,     0,     0]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]])},)

In [318]:
outputs = bert(input_ids = sample["input_ids"][:2], attention_mask = sample["attention_mask"][:2])
print(outputs.last_hidden_state.shape) # (bs, seq_length, hidden_dim)
type(outputs)

torch.Size([2, 512, 128])


transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [320]:
import torch.nn as nn
W = nn.Linear(bert.config.hidden_size, 2)

h_cls = outputs.last_hidden_state[:,0]
logits = W(h_cls)
logits.shape, logits

(torch.Size([2, 2]),
 tensor([[-0.1802,  0.9329],
         [-0.7781,  0.3020]], grad_fn=<AddmmBackward0>))

In [323]:
softmax = torch.nn.Softmax(dim=0)
logits, softmax(logits).tolist()

(tensor([[-0.1802,  0.9329],
         [-0.7781,  0.3020]], grad_fn=<AddmmBackward0>),
 [[0.645167887210846, 0.6526991128921509],
  [0.35483217239379883, 0.34730085730552673]])

In [45]:
for i in range(5):
    print("Outer:", i)
    for j in range(5):
        print("  Inner:", j)
        if j == 2:
            break


Outer: 0
  Inner: 0
  Inner: 1
  Inner: 2
Outer: 1
  Inner: 0
  Inner: 1
  Inner: 2
Outer: 2
  Inner: 0
  Inner: 1
  Inner: 2
Outer: 3
  Inner: 0
  Inner: 1
  Inner: 2
Outer: 4
  Inner: 0
  Inner: 1
  Inner: 2


In [295]:
from model import ColaModel 
from data import DataModule

processor = DataModule()

text = train_data[0]["sentence"]
print(text)
inference_sample = {"sentence": text}
print(type(inference_sample))
processed = processor.tokenize_data(inference_sample)
print(processed.keys())
print(type(processed["input_ids"]))

bert = AutoModel.from_pretrained(model_name)

logits = bert(
    torch.tensor(processed["input_ids"], dtype=torch.long),
    torch.tensor(processed["attention_mask"], dtype=torch.long),
) # (bs, 2)

logits




Our friends won't buy this analysis, let alone the next one we propose.
<class 'dict'>
Processing batch of size: 71
KeysView({'input_ids': [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

ValueError: not enough values to unpack (expected 2, got 1)

In [287]:
import torch.nn as nn 



bert = AutoModel.from_pretrained(model_name)
# bert.config
W = nn.Linear(bert.config.hidden_size, 2)

def forward(input_ids, attention_mask):
    outputs = bert(input_ids=input_ids, attention_mask=attention_mask)
    # print(vars(outputs).keys()) # to see only attributes (not methods) in outputs

    # last_hidden_state is (bs, seq_length, hidden_dim)
    h_cls = outputs.last_hidden_state[:,0] # (bs, hidden_dim)
    logits = W(h_cls) # (bs, 2)
    return logits

logits = forward(processed["input_ids"], processed["attention_mask"])
logits


<class 'dict'>
Processing batch of size: 71


AttributeError: 'list' object has no attribute 'size'

In [285]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-1): 2 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [267]:
hparams

NameError: name 'hparams' is not defined

### inference

In [44]:
a = torch.Tensor([-0.4495,  0.3664])
b = torch.nn.functional.softmax(a, dim=0)
# c = torch.nn.functional.softmax(a, dim=1)
a, b, a.shape,

(tensor([-0.4495,  0.3664]), tensor([0.3066, 0.6934]), torch.Size([2]))

In [37]:
a = torch.Tensor([[-0.4495,  0.3664]])
print(a.shape)
b = torch.nn.functional.softmax(a, dim=0)
c = torch.nn.functional.softmax(a, dim=1)
a, b, c


torch.Size([1, 2])


(tensor([[-0.4495,  0.3664]]), tensor([[1., 1.]]), tensor([[0.3066, 0.6934]]))

In [24]:
import torch
from model import ColaModel
from data import DataModule

class ColaPredictor:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = ColaModel.load_from_checkpoint(model_path)
        self.model.eval()
        self.model.freeze()
        self.processor = DataModule()
        self.softmax = torch.nn.Softmax(dim=0)
        self.labels = ["unacceptable", "acceptable"]
        self.device = next(self.model.parameters()).device
    
    def predict(self, text):
        inference_sample = {"sentence": text}
        processed = self.processor.tokenize_data(inference_sample)

        
        input_ids = torch.tensor([processed["input_ids"]], dtype=torch.long).to(self.device)
        attention_mask = torch.tensor([processed["attention_mask"]], dtype=torch.long).to(self.device)

        logits = self.model(input_ids, attention_mask)
        print(logits.shape)
        scores = self.softmax(logits[0]) #(bs,)
        print(scores)
        predictions = []
        for score, label in zip(scores, self.labels):
            predictions.append({"label": label, "score": score})
        print(predictions)
        return predictions

In [26]:
sentence = "The boy is sitting on a bench"
predictor = ColaPredictor("./models/epoch=3-step=1072.ckpt")

In [27]:
print(predictor.predict(sentence))

{'sentence': 'The boy is sitting on a bench'}
The boy is sitting on a bench
Processing batch of size: 29
torch.Size([1, 2])
tensor([0.3066, 0.6934], device='cuda:0')
[{'label': 'unacceptable', 'score': tensor(0.3066, device='cuda:0')}, {'label': 'acceptable', 'score': tensor(0.6934, device='cuda:0')}]
[{'label': 'unacceptable', 'score': tensor(0.3066, device='cuda:0')}, {'label': 'acceptable', 'score': tensor(0.6934, device='cuda:0')}]
